# Derivace a modely založené na derivaci

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Numerická simulace v rovnici ochlazování

Studujeme úlohu $$\frac{\mathrm dT}{\mathrm dt} = - k (T-T_\infty), \quad T(0)=T_0.$$ Pokusíme se odhadnout po malých krůčcích chování řešení. Uvnitř každého krůčku budeme rychlost považovat za konstantní a změnu teploty určíme jako součin rychlosti a času (délky časového intervalu).

### Srovnání přesného analytického řešení s numerickou simulací po krocích konečné délky.

In [ ]:
# Nastavení parametrů
tmin, tmax = 0,10
dt = 0.5  # delka kroku
k = 0.5
T0 = 100
T_inf = 20

# Pomocné proměnné
N = int((tmax-tmin)/dt+1) # počet bodů pro simulaci
t = np.linspace(tmin,tmax,N) # časová osa pro simlaci

# Počáteční nastavení
T = np.zeros(N)  # pole pro ukládání teplot
T[0] = T0  # počáteční teplota

# Simulace po časových krocích
for i in range(1,N):
    rozdil_teplot = T[i-1] - T_inf
    rychlost_ochlazovani = k*rozdil_teplot
    zmena_teploty = - dt*rychlost_ochlazovani
    T[i] = T[i-1] + zmena_teploty

# Uložení do tabulky
df = pd.DataFrame()
df["t"] = t
df["numericky"] = T
df["analyticky"] = T_inf + (T0-T_inf)*np.exp(-k*t)   
    
# Vykreslení tabulky    
df.plot(x="t")
plt.grid()
plt.gca().set(
    xlabel="čas",
    ylabel="teplota",
    title="Ochlazování rychlostí úměrnou teplotnímu rozdílu",
    ylim=(0,None)
);


**Úkol** Experimentujte s délkou kroku `dt` a sledujte hladkost numerického řešení a jeho odchylku od přesného analytického řešení. 

V praxi musíme mít krok dostatečně malý, aby řešení bylo hladké a přesné, ale ne moc malý, aby nás to nestálo hodně paměti, výpočetního výkonu, času a aby nehrozilo, že se při mnoha výpočtech akumulují zaokrouhlovací chyby. Zpravidla nám toto obstarají procedury pro řešení automaticky.

### Simulace pro více počátečních podmínek

Vyjdeme z předchozí simulace, ale novou simulaci spustíme v cyklu. Jeden průběh cyklu pro každou počáteční podmínku. **Recyklujeme kód z předchozí buňky a změny označíme vykřičníkem.**

In [ ]:
# Nastavení parametrů
tmin, tmax = 0,10
dt = 0.1 # !!! zmena, jemnejsi krok
k = 0.5
T0_seznam = [100, 80, 60, 40]  # !!! zmena
T_inf = 20

# Pomocné proměnné
N = int((tmax-tmin)/dt+1) # počet bodů pro simulaci
t = np.linspace(tmin,tmax,N) # časová osa pro simlaci

# Uložení do tabulky   !!! Zmena, tabulka pred simulaci
df = pd.DataFrame()
df["t"] = t

for T0 in T0_seznam:
    # Počáteční nastavení
    T = np.zeros(N)  # pole pro ukládání teplot
    T[0] = T0  # počáteční teplota

    # Simulace po časových krocích
    for i in range(1,N):
        rozdil_teplot = T[i-1] - T_inf
        rychlost_ochlazovani = k*rozdil_teplot
        zmena_teploty = - dt*rychlost_ochlazovani
        T[i] = T[i-1] + zmena_teploty

    df[T0] = T # !!! zmena, dalsi sloupec tabulky

# Tisk začátku tabulky    !!!
print(df.head())    

# Vykreslení tabulky    
df.plot(x="t")
plt.grid()
plt.gca().set(
    xlabel="čas",
    ylabel="teplota",
    title="Ochlazování rychlostí úměrnou teplotnímu rozdílu",
    ylim=(0,None)
)
plt.legend(title="Počáteční podmínka"); # !!! nadpis pro legendu

#### Úkol - simulace pro měnící se koeficient úměrnosti

Udělejte simulaci s jednou počáteční podmínkou a měnící se hodnotu koeficientu $k$. Nakopírujte si sem předchozí buňku a modifikujte tak, aby byla počáteční podmínka pořád stejná a měnil se koeficient $k$. Musíte sestavit seznam hodnot pro proměnnou $k$, změnit cyklování a změnit jméno sloupce v tabulce tak, aby zachycovalo hodnotu $k$.

### Simulace pro model se zpožděním

Uvedené simulace jsou naivní metodou řešení úlohy modelující pomocí derivací nějaký proces. Pro řešení modelů s derivacemi existují mnohem vyspělejší metody, které si představíme příště. V praxi se zpravidla spoléháme na tyto naprogramované profesionály. Dovednost umět si implementovat jednoduchý řešič může sloužit pro orientační první nástřel výpočtu, nebo při potřebě řešení nějaké speciální úlohy. Jedním takovým speciálním postupem může být snaha započítat do rovnice ochlazování zpoždění. To by mohlo odpovídat regulaci teploty, kdy se změna v nastavení neprojeví okamžitě. Všichni takovou situaci známe při nastavování teploty ve sprše.

In [ ]:
# Nastavení parametrů
tmin, tmax = 0,20
dt = 0.01 # !!! zmena, jemnejsi krok
k = 0.5
T0 = 100
zpozdeni_seznam = [0, 10, 60, 100, 200]  # !!! zmena, zpozdeni v jednotkach delky kroku
T_inf = 20

# Pomocné proměnné
N = int((tmax-tmin)/dt+1) # počet bodů pro simulaci
t = np.linspace(tmin,tmax,N) # časová osa pro simlaci

# Uložení do tabulky   
df = pd.DataFrame()
df["t"] = t

for zpozdeni in zpozdeni_seznam:  # !!! zmena v cyklu
    # Počáteční nastavení
    T = np.zeros(N)  # pole pro ukládání teplot
    T[0] = T0  # počáteční teplota

    # Simulace po časových krocích
    for i in range(1,N):
        rozdil_teplot = T[max(i-1-zpozdeni,0)] - T_inf  # !!! zmena, zapocitani zpozdeni
        rychlost_ochlazovani = k*rozdil_teplot
        zmena_teploty = - dt*rychlost_ochlazovani
        T[i] = T[i-1] + zmena_teploty

    df[zpozdeni] = T # !!! zmena, dalsi sloupec tabulky
    
# Vykreslení tabulky    
df.plot(x="t")
plt.grid()
plt.gca().set(
    xlabel="čas",
    ylabel="teplota",
    title="Ochlazování rychlostí úměrnou teplotnímu rozdílu",
    ylim=(0,None)
)
plt.legend(title="Zpoždění");